#Shenandoah or G1GC: which is faster for batch loading www.uniprot.org data?

www.uniprot.org is one of the wider used websites in the life-sciences. The part being benchmarked here is the loading of a representative subset of all the uniprot data into a custom key-value store and indexed by lucene. This exercises a lot of code (including native processes and JNI) and is mostly parallel with serial bottlenecks at different points. i.e. it's over 200,00 lines of in house plus another 125 jars with other stuff ;)

The code used was nightly build 125 from shiplev.net (https://builds.shipilev.net/openjdk-shenandoah-jdk8/). The same code is used for both the G1GC and Shenandoah tests. The benchmark runs are interleaved back to back. i.e. we run a G1GC run then a Shenandoah run. Only differences are name of log file pattern and -XX:UseShenandoahGC or -XX:UseG1GC

The hardware is dell precision 5510 running fedora 27 with 16GB of ram a 256GB xfs NVMe and i7-6700HQ CPU @ 2.60GHz
While there is a good reason not to test on laptops for short benchmarks they dissapear when the benchmark runs for nearly 3 days. Testing on a laptop is ok but assumes no one will close the lid  while the benchmarker is not looking ;)
That is why there are 49 Shenandoah runs, and 50 G1GC runs. One Shenandoah run was removed as outlier due to the machine being put in sleep mode. 

In [1]:
library(lubridate)

Shenandoah <- read.csv(file="./shenandoah-build-125-jdk8.csv", header=FALSE, sep=",")
G1GC <- MyData <- read.csv(file="./g1gc-build-125-jdk8.csv", header=FALSE, sep=",")

Sseconds <- unlist(lapply(lapply(Shenandoah$V1, hms), as.numeric))
Gseconds <- unlist(lapply(lapply(G1GC$V1, hms), as.numeric))
summary(Sseconds)
summary(Gseconds)

var.test(Sseconds, Gseconds)

#test if Sseconds is likely to be less (lower) than Gseconds
t.test(Sseconds,Gseconds, alternative="less")
#test if Gseconds is likely to be less (lower) than Sseconds, sanity check
t.test(Gseconds,Sseconds, alternative="less")

time_length((sum(Sseconds) +sum(Gseconds)), unit = "days")


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date



   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   2246    2299    2314    2317    2337    2418 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   2364    2418    2431    2441    2460    2545 


	F test to compare two variances

data:  Sseconds and Gseconds
F = 1.1134, num df = 48, denom df = 49, p-value = 0.709
alternative hypothesis: true ratio of variances is not equal to 1
95 percent confidence interval:
 0.6304874 1.9696914
sample estimates:
ratio of variances 
          1.113449 



	Welch Two Sample t-test

data:  Sseconds and Gseconds
t = -15.376, df = 96.471, p-value < 2.2e-16
alternative hypothesis: true difference in means is less than 0
95 percent confidence interval:
    -Inf -110.34
sample estimates:
mean of x mean of y 
 2316.913  2440.614 



	Welch Two Sample t-test

data:  Gseconds and Sseconds
t = 15.376, df = 96.471, p-value = 1
alternative hypothesis: true difference in means is less than 0
95 percent confidence interval:
     -Inf 137.0633
sample estimates:
mean of x mean of y 
 2440.614  2316.913 


[1] 2.726383

T-test shows that the eye observation of Shenandoah being faster than G1GC holds. The inverse test shows the result is sane.

In our experience UseParallelGC is significantly slower than G1GC for this workload. 
The improvement of Shenandoah is roughly equivalent to upgrading from JDK8 to JDK9 and such a 4-5% improvement in performance normally costs us two to three weeks of coding.

#Shendoah/Parallel JDK10 build-126-x86-64-release

This is testing a nightly for both Shenandoah and ParallelGC again from https://builds.shipilev.net/

In [8]:
library(lubridate)

Shenandoah <- read.csv(file="./shenandoah-build-126-jdk10.csv", header=FALSE, sep=",")
Parallel <- MyData <- read.csv(file="./parallelgc-build-126-jdk10.csv", header=FALSE, sep=",")

Sseconds <- unlist(lapply(lapply(Shenandoah$V1, hms), as.numeric))
Pseconds <- unlist(lapply(lapply(Parallel$V1, hms), as.numeric))
summary(Sseconds)
summary(Pseconds)

var.test(Sseconds, Pseconds)

#test if Sseconds is likely to be less (lower) than Gseconds
t.test(Sseconds,Pseconds, alternative="less")
#test if Gseconds is likely to be less (lower) than Sseconds, sanity check
t.test(Pseconds,Sseconds, alternative="less")

time_length((sum(Sseconds) +sum(Pseconds)), unit = "days")

OrderedSseconds <- sort(Sseconds)
OrderedPseconds <- sort(Pseconds)


plot(OrderedSseconds, type="l", col="red", main="Runtimes of uniprot test workload with different GC", ylab="time")
    lines(OrderedPseconds, col="green")
    


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1853    1894    1904    1910    1929    2038 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1858    1879    1889    1892    1901    1959 


	F test to compare two variances

data:  Sseconds and Pseconds
F = 2.567, num df = 33, denom df = 33, p-value = 0.008282
alternative hypothesis: true ratio of variances is not equal to 1
95 percent confidence interval:
 1.282032 5.139943
sample estimates:
ratio of variances 
          2.567016 



	Welch Two Sample t-test

data:  Sseconds and Pseconds
t = 2.4675, df = 55.323, p-value = 0.9916
alternative hypothesis: true difference in means is less than 0
95 percent confidence interval:
     -Inf 29.66454
sample estimates:
mean of x mean of y 
 1909.572  1891.893 



	Welch Two Sample t-test

data:  Pseconds and Sseconds
t = -2.4675, df = 55.323, p-value = 0.008364
alternative hypothesis: true difference in means is less than 0
95 percent confidence interval:
      -Inf -5.693398
sample estimates:
mean of x mean of y 
 1891.893  1909.572 


[1] 1.495947